# Q-Table Learning for FrozenLake

This is an implementation of Q-Table learning for the reinforcement learning environment FrozenLake. This follows the tutorial from [awjuliani](https://gist.github.com/awjuliani/9024166ca08c489a60994e529484f7fe#file-q-table-learning-clean-ipynb).  

## The Environment

In [1]:
import gym
import numpy as np

In [2]:
env = gym.make('FrozenLake-v0')

`env` is a $4 \times 4$ grid, with four types of tiles: S (starting), F (frozen), H (hole), and G (goal). The agent starts on the S tile and can choose one of four directions to move - 0 (left), 1 (down), 2 (right), and 3 (up). The agent must make it to the G tile while avoiding the H tile. Here's the environment.

In [3]:
env.reset()
env.render()


SFFF
FHFH
FFFH
HFFG


A run ends when the agent falls into an H tile (reward: 0) or the agent finds the goal (reward: 1). 

The catch: the frozen lake is slippery. When the agent chooses to go in a specific direction, there's only a 1/3 chance the agent will actually go in that direction. There's a 2/3rd chance that the agent will go perpendicular to the desired direction (1/3rd to the left, with respect to the desired direction, and 1/3rd to the right). Because of this schotastic element the environment cannot be perfectly solved. 

In fact, when you look at the finished Q-table as a sanity check, you'll find that the best strategy to take when located next to an H tile is to move *directly away from it,* even if that doesn't get you closer to the goal. This is the one way the agent can guarantee that it *won't* fall into the hole (except when you are adjacent to two holes, which occurs once, at row 2, column 3). I had to check the source code to verify the slippery behaviour; before I did that I was confused by the strategies learned by the Q-Table. 

The locations are index from 0 to 15, going along rows, then by columns.

The Q-Table contains Q-values for state-action pairs. The Q-value for a state-action pair is the *estimated, expected value* of the reward for taking action $a$ when currently in state $s$. It takes the following form:

$Q(s, a) = r + \gamma(\text{max}(Q(s', a'))$

It's equal to the reward received haven entered state $s$ as well as the maximum discounted Q-value for the possible actions taken from this state. Each Q-value is updated according to Bellman's equation:

$Q(s, a) \leftarrow Q(s, a) + \alpha (r + \gamma \text{max}(Q(s', a')) - Q(s, a))$

Where $\alpha$ is the learning rate. 

## Q-Table Learning

In [4]:
Q = np.zeros([env.observation_space.n, env.action_space.n])
lr = .8
y = .95
num_episodes = 5000
rList = []
for i in range(num_episodes):
    s = env.reset()
    rAll = 0
    d = False
    j = 0
    while j < 99:
        j += 1
        a = np.argmax(Q[s,:] + np.random.randn(1, env.action_space.n) * (1./(i+1)))
        s1,r,d,_ = env.step(a)
        Q[s,a] = Q[s,a] + lr*(r + y*np.max(Q[s1,:]) - Q[s,a])
        rAll += r
        s = s1
        if d == True:
            break
    rList.append(rAll)

In [5]:
print("Average reward per episode: " + str(sum(rList)/num_episodes))

Average reward per episode: 0.5496


In [6]:
print("Q Table values")
print(np.round(Q, 3))

Q Table values
[[0.217 0.007 0.009 0.   ]
 [0.002 0.004 0.002 0.151]
 [0.002 0.002 0.006 0.242]
 [0.    0.001 0.    0.068]
 [0.346 0.    0.001 0.002]
 [0.    0.    0.    0.   ]
 [0.017 0.    0.    0.   ]
 [0.    0.    0.    0.   ]
 [0.001 0.    0.003 0.225]
 [0.    0.235 0.002 0.002]
 [0.116 0.    0.    0.   ]
 [0.    0.    0.    0.   ]
 [0.    0.    0.    0.   ]
 [0.    0.    0.202 0.001]
 [0.    0.    0.    0.577]
 [0.    0.    0.    0.   ]]


In [7]:
dic = {0:'<', 1:'▼', 2:'>', 3:'▲'}

In [8]:
out = []
for state, actions in enumerate(np.round(Q, 3)):
    if np.sum(actions) == 0:
        out.append('O')
    else:
        out.append(dic[np.argmax(actions)])
out[-1] = '!'
out = np.array(out)
out = out.reshape(4, 4)
print(out)

[['<' '▲' '▲' '▲']
 ['<' 'O' '<' 'O']
 ['▲' '▼' '<' 'O']
 ['O' '>' '▲' '!']]


We can see that the learner has learned to be "afraid" of holes, whenever it is adjacent to one the optimal behaviour is to move directly away from it. There's trouble at row 2, column 3 because moving left or right will put the agent in a hole. Interestingly, in this situation, the agent usually learns the optimal action is to move towards one of the holes. This is, in fact, the best strategy, because if the agent moves up or down directly toward safe ground, it actually has a 2/3rd chance of slipping sideways into a hole, whereas if it goes directly toward either of the holes it has a 2/3rd chance of slipping into safety. 

# Q-Learning with Tensorflow and Neural Networks

Alright, let's do this with a neural network that determines the best action to take. 

The input to the neural network is the state - the location of the agent - which is represented in as a one-hot vector. The output is a vector of 4 Q-values associated with each of the possible actions to take. The loss function is a sum-of-squares loss, derived from the difference between the network's predicted Q-values and the target Q-values. 

In [9]:
import numpy as np
import random
import tensorflow as tf
import matplotlib.pyplot as plt

In [10]:
env = gym.make('FrozenLake-v0')

In [11]:
tf.reset_default_graph()

In [12]:
inputs1 = tf.placeholder(shape=[1, 16], dtype=tf.float32)
W = tf.Variable(tf.random_uniform([16,4],0,0.01))
Qout = tf.matmul(inputs1, W)
predict = tf.argmax(Qout, 1)

nextQ = tf.placeholder(shape=[1,4], dtype=tf.float32)
loss = tf.reduce_sum(tf.square(nextQ-Qout))
trainer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
updateModel = trainer.minimize(loss)

In [13]:
init = tf.global_variables_initializer()
y = 0.99
e = 0.1 # epsilon, chance of taking a random action
num_episodes = 2000

jList = []
rList = []

with tf.Session() as sess:
    sess.run(init)
    for i in range(num_episodes):
        s = env.reset()
        rAll = 0
        d = False
        j = 0
        while j < 99:
            j += 1
            # np.identity makes an identity matrix; 1s along diagonal
            # get one-hot encoding by selecting the sth row
            # use list slicing to get a 1 x 16 encoding
            a, allQ = sess.run([predict, Qout], feed_dict = {inputs1: np.identity(16)[s:s+1]})
            if np.random.rand(1) < e:
                a[0] = env.action_space.sample()
            s1, r, d, _ = env.step(a[0])
            # Q value for this state
            Q1 = sess.run(Qout, feed_dict={inputs1:np.identity(16)[s1:s1+1]})
            # get the max
            maxQ1 = np.max(Q1)
            targetQ = allQ
            targetQ[0, a[0]] = r + y * maxQ1
            _, W1 = sess.run([updateModel, W], feed_dict={inputs1:np.identity(16)[s:s+1], nextQ:targetQ})
            rAll += r
            s = s1
            if d == True:
                # decrease epsilon as training occurs
                e = 1./((i/50) + 10)
        jList.append(j)
        rList.append(rAll)
        if i % 100 == 0:
            print("{} episodes completed.".format(i))

0 episodes completed.
100 episodes completed.
200 episodes completed.
300 episodes completed.
400 episodes completed.
500 episodes completed.
600 episodes completed.
700 episodes completed.
800 episodes completed.
900 episodes completed.
1000 episodes completed.
1100 episodes completed.
1200 episodes completed.
1300 episodes completed.
1400 episodes completed.
1500 episodes completed.
1600 episodes completed.
1700 episodes completed.
1800 episodes completed.
1900 episodes completed.


In [14]:
print("Average reward per episode: {}".format(sum(rList)/num_episodes))

Average reward per episode: 0.439
